In [ ]:
# 현장에서 많이씀(성능이 좋기때문) => RandomForest

In [ ]:
params 
KNN { 'n_neighbors' : range(3,51,2), 'metric' : ['euclidean', 'manhattan']  }
DecisionTree { 'max_depth':range(2,11), 'min_samples_leaf':range(10,101,10) }
RandomForest { 'n_estimators':[20,50,100], 'max_features':range(1,21) }
XGBoost {'learning_rate' : np.linspace(0.01,0.2, 20), 'n_estimators':range(60,200,20), 'max_depth':[3,4,5,6]}
SVM { 'C' : np.linspace(0.01, 100, 50), 'gamma':[0.001,0.01,.1,1] }

GridSearchCV(m, params, cv=5, scoring = 'neg_mean_absolute_error')
XGBRegressor(objective = 'reg:squarederror')

## 라이브러리

In [1]:
#라이브러리들을 불러오자.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 전처리
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 모델링
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import * 

# 비지도 학습
from sklearn.datasets import make_blobs, make_moons
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans, DBSCAN

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

import warnings
warnings.filterwarnings(action='ignore')

## target 변수 조회

In [ ]:
print(data['Creditability'].value_counts())
print(data['Creditability'].value_counts()/ data.shape[0])

data['Creditability'].value_counts().plot(kind = 'barh')
plt.show()

## 데이터 준비

In [ ]:
# 데이터분할1
target = 'Sales'
x = data.drop(target, axis=1)
y = data.loc[:, target]

# 가변수화
dumm_cols = ['ShelveLoc','Education','Urban', 'US']
x = pd.get_dummies(x, columns = dumm_cols, drop_first = True)

# 데이터 분할2
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=.2
                                                  , random_state = 2022)

# 스케일링
scaler = MinMaxScaler()
x_train_s = scaler.fit_transform(x_train)
x_val_s = scaler.transform(x_val)

x_train_s = pd.DataFrame(x_train_s, columns=list(x))
x_val_s = pd.DataFrame(x_val_s, columns=list(x))

## Fitting Graph ->Elbow Method [모델과적합]

* 가장 단순한 모델(평균모델)
** knn : k를 최대로 크게하면 평균 모델이 됨.
** k의 최대값은 학습 데이터의 행 수

* Decision Tree
** Decision Tree는 나무의 크기가 클 수록 복잡한 모델
** 크기를 결정하는 파라미터는 max_depth

In [ ]:
# 모델링
n = x_train_s.shape[0]
model = KNeighborsRegressor(n_neighbors = n) # train set의 행 수
model.fit(x_train_s, y_train)
pred_train = model.predict(x_train_s)

In [ ]:
result_train = [] # train set을 가지고 예측한 결과
result_val = [] # val set을 가지고 예측한 결과
k_values = list(range(1,101))

# KNN
for d in k_values :
    model = KNeighborsClassifier(n_neighbors= d)
    model.fit(x_train_s, y_train)
    pred_tr, pred_val = model.predict(x_train_s), model.predict(x_val_s)
    result_train.append(accuracy_score(y_train, pred_tr))
    result_val.append(accuracy_score(y_val, pred_val))
    
result_train = [] # train set을 가지고 예측한 결과
result_val = [] # val set을 가지고 예측한 결과
depth = list(range(1,21))

# Decision Tree
for d in depth :
    model = DecisionTreeClassifier(max_depth = d)
    model.fit(x_train, y_train)
    pred_tr, pred_val = model.predict(x_train), model.predict(x_val)
    result_train.append(accuracy_score(y_train, pred_tr))
    result_val.append(accuracy_score(y_val, pred_val))

In [ ]:
plt.figure(figsize = (12,8))
plt.plot(k_values, result_train, label = 'train_acc', marker = 'o')
plt.plot(k_values, result_val, label = 'val_acc', marker = 'o')

plt.xlabel('Complexity')
plt.ylabel('Accuracy')
plt.legend()
plt.grid()
plt.show()

## Decision Tree[ 변수중요도 그래프 ] 실전에서 많이 씀

In [ ]:
# 이 이름을 이용하여 시각화
plt.figure(figsize = (20,8)) # 그림 사이즈 조절
plot_tree(m3.best_estimator_, feature_names = x_train.columns, 
               class_names= ['Bad', 'Good'], filled = True, fontsize = 10);

In [ ]:
def plot_feature_importance(importance, names):
    feature_importance = np.array(importance)
    feature_names = np.array(names)

    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)

    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)
    fi_df.reset_index(drop=True, inplace = True)

    plt.figure(figsize=(10,8))
    sns.barplot(x='feature_importance', y='feature_names', data = fi_df)

    plt.xlabel('FEATURE IMPORTANCE')
    plt.ylabel('FEATURE NAMES')
    plt.grid()

    return fi_df

In [ ]:
result = plot_feature_importance(model.feature_importances_, list(x_train))

In [ ]:
result = plot_feature_importance(m3.best_estimator_.feature_importances_, list(x_train)) # 튜닝했기 때문에

## Regessor 차트 확인

In [ ]:
# 튜닝 과정 로그를 df로 저장 합시다.
result = pd.DataFrame(m1_gs.cv_results_)

# 튜닝 결과를 그래프로 그려봅시다.
plt.figure(figsize = (10,6))
sns.lineplot(x='param_max_depth', y='mean_test_score', data = result)  # plt.plot
plt.grid()
plt.show()

In [ ]:
SVR(Regressor)
# 이를 차트로 그려봅시다.
plt.figure(figsize = (12,8))
sns.lineplot(x = 'param_C', y = 'mean_test_score', data = result, hue = 'param_gamma')
plt.grid()
plt.show()

## XGB 변수중요도 / 시각화

In [ ]:
#위에꺼 plot_feature_importance 함수 가져오기

In [ ]:
# xgb의 변수 중요도
weight : 모델 전체에서 해당 feature가 split될 때 사용된 횟수의 합(plot_tree 에서의 기본값)
gain : feature별 평균 imformation gain.(model.feature_importances_ 의 기본값)
cover : feature가 split 할때의 샘플 수의 평균.

In [ ]:
from xgboost import XGBRegressor, plot_tree, plot_importance
plt.rcParams['figure.figsize'] = 8, 5
plot_importance(model)
plt.show()

In [ ]:
result = plot_feature_importance(model.feature_importances_, list(x),6) #x_train

In [ ]:
# 시각화(Tree하나씩 열어볼 수 있음)
plt.rcParams['figure.figsize'] = 20,20  # 파일전체에 영향을 미침 !!!!
plot_tree(model, num_trees = 0) 
plt.show()

# xgboost 자체 plot_tree 함수를 제공합니다.
# plot_tree(model, num_trees = 0)
# num_trees : 전체 트리 5개짜리 모델이므로 각각 0~4까지 인덱스로 조회해 볼 수 있다.

## 성능비교

In [ ]:
RMSE, MAE, MAPE = [],[],[]
model_desc = ['lr_selected', 'lr_all','knn','dt','rf','xgb','svm']
pred = [p1, p2, p3, p4, p5, p6,p7]

for i, p in enumerate(pred) :
    RMSE.append(mean_squared_error(y_val, p, squared=False))
    MAE.append(mean_absolute_error(y_val, p))
    MAPE.append(mean_absolute_percentage_error(y_val, p))

result = pd.DataFrame({'model_desc':model_desc,'RMSE':RMSE,'MAE':MAE,'MAPE':MAPE})
result

In [ ]:
print(confusion_matrix(y_val, pred1))
print(classification_report(y_val, pred1))

## Decision Tree 시각화 및 변수중요도

In [ ]:
plt.figure(figsize = (10,10)) # 그림 사이즈 조절
plot_tree(m1, feature_names = list(x_train), 
               class_names= ['Stay', 'Leave'], filled = True, fontsize = 10);  # class_names = [target의 내용]
plt.show()
print('-'*88)
# 변수 중요도
print(list(x_train))
print(m1.feature_importances_)
print('-'*88)
print(classification_report(y_val, p1))

## 로지스틱 회귀를 위한 전진선택법 함수 

In [ ]:
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant

def forward_stepwise_linear(x_train, y_train):

    # 변수목록, 선택된 변수 목록, 단계별 모델과 AIC 저장소 정의
    features = list(x_train)
    selected = []
    step_df = pd.DataFrame({ 'step':[], 'feature':[],'aic':[]})

    # 
    for s in range(0, len(features)) :
        result =  { 'step':[], 'feature':[],'aic':[]}

        # 변수 목록에서 변수 한개씩 뽑아서 모델에 추가
        for f in features :
            vars = selected + [f]
            x_tr = x_train[vars]
            model = OLS(y_train, add_constant(x_tr)).fit(disp=False)
            result['step'].append(s+1)
            result['feature'].append(vars)
            result['aic'].append(model.aic)
        
        # 모델별 aic 집계
        temp = pd.DataFrame(result).sort_values('aic').reset_index(drop = True)

        # 만약 이전 aic보다 새로운 aic 가 크다면 멈추기
        if step_df['aic'].min() < temp['aic'].min() :
            break
        step_df = pd.concat([step_df, temp], axis = 0).reset_index(drop = True)

        # 선택된 변수 제거
        v = temp.loc[0,'feature'][s]
        features.remove(v)

        selected.append(v)
    
    # 선택된 변수와 step_df 결과 반환
    return selected, step_df

In [ ]:
vars, result = forward_stepwise_logistic(x_train, y_train)

In [ ]:
# 선택된 변수
lr_m1 = LinearRegression()
lr_m1.fit(x_train[vars], y_train)
p1 = lr_m1.predict(x_val[vars])

print('RMSE : ', mean_squared_error(y_val, p1, squared=False))
print('MAE  : ', mean_absolute_error(y_val, p1))
print('MAPE : ', mean_absolute_percentage_error(y_val, p1))

## cross_val_score

In [ ]:
from sklearn.model_selection import cross_val_score
# train + validation set을 이용하여 학습, 예측, 평가를 한번에. (여기서는 .fit 이 아님!)
dt_result = cross_val_score(model, x, y, cv=10)
print(dt_result)
print(dt_result.mean(), dt_result.std())

## Permutation Feature Importance(그 외 변수중요도 그래프)

In [ ]:
from sklearn.inspection import permutation_importance

In [ ]:
pfi1 = permutation_importance(model4, x_val_s, y_val, n_repeats=10, scoring = 'r2', random_state=2022)
# scoring = 'accuracy' default[분류 모델]
# deep learning 모델에 대해서는 명시적으로 scoring = 'r2'을 지정해 필요

In [ ]:
# feature별 Score 분포
plt.figure(figsize = (10,8))
for i,vars in enumerate(list(x)) :
    sns.kdeplot(pfi1.importances[i], label = vars)
plt.grid()
plt.legend()
plt.show()

In [ ]:
sorted_idx = pfi1.importances_mean.argsort()
plt.figure(figsize = (10, 8))
plt.boxplot(pfi1.importances[sorted_idx].T, vert=False, labels=x.columns[sorted_idx])
plt.axvline(0, color = 'r')
plt.grid()
plt.show()

In [ ]:
# 평균값으로 변수중요도 그래프 그리기
pfi1 = permutation_importance(model1, x_val_s, y_val, n_repeats=10, scoring = 'r2', random_state=2022)

In [ ]:
# 변수 중요도 plot(가져오기)
result = plot_feature_importance(pfi1.importances_mean, list(x_train))

## Partial Dependence Plot(개별 변수별 관계)

* 변수 중요도,PDP : train 데이터로 살펴보는 것이 기본이다. 그리고 결과가 유사해야 한다. 

In [ ]:
from sklearn.inspection import plot_partial_dependence, partial_dependence

In [ ]:
# 개별 변수 분석
var = 'MonthlyIncome'

plt.rcParams['figure.figsize'] = 10, 6
plot_partial_dependence(model, features = [var], X = x_val)
plt.grid()
plt.show()

In [ ]:
# 전체 데이터로 확인
var = 'MonthlyIncome'

plt.rcParams['figure.figsize'] = 12, 8
plot_partial_dependence(model, features = [var], X = x_val, kind = 'both') #kind = 'average'
plt.grid()
plt.show()

In [ ]:
# 두개 변수 비교
plot_partial_dependence(model, features = ['rm','lstat'], X = x_train)
plt.show()

In [ ]:
# 두개 변수 비교
plot_partial_dependence(model2, features = [('CreditAmount','Age')], X = x_val)
plt.show()

In [ ]:
# 스케일링한거 데이터 프레임에 넣어서 뽑아야함
x_train_s = pd.DataFrame(x_train_s, columns = list(x))  # 칼럼이름 지정 필요!!
x_val_s = pd.DataFrame(x_val_s, columns = list(x))

## SHAP 값으로 모델의 예측 설명하기

In [ ]:
# 선형회귀는 회귀계수로 변수 기여도 해석해도 무방

In [ ]:
# (회귀모델)
explainer = shap.TreeExplainer(model)
shap_values = explainer1.shap_values(x_train)

# (분류모델)
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(x)

In [ ]:
# 예측하기 위해서는 입력데이터(x)가 2차원이어야 합니다.
pred = model1.predict(x_train.iloc[0:1,:])
pred

In [ ]:
# train의 평균(회귀)
explainer1.expected_value

# train의 평균(분류)
explainer1.expected_value[1]

In [ ]:
# 인덱스별 데이터
shap.initjs() # javascript 시각화 라이브러리 --> colab에서는 모든 셀에 포함시켜야 함.-> 아나콘다에서는 한번만 !
index=371
# force_plot(전체평균, shapley_values, input)
shap.force_plot(explainer1.expected_value, shap_values1[index,:], x.iloc[index,:])

# 분류모델
shap.initjs() # javascript 시각화 라이브러리 --> colab에서는 모든 셀에 포함시켜야 함.-> 아나콘다에서는 한번만 !
index=932
# force_plot(전체평균, shapley_values, input)
shap.force_plot(explainer.expected_value[1], shap_values[1][index,:], x.iloc[index,:])

In [ ]:
# 전체 데이터
shap.initjs() # javascript 시각화 라이브러리 --> colab에서는 모든 셀에 포함시켜야 함.-> 아나콘다에서는 한번만 !

# force_plot(전체평균, shapley_values, input)
shap.force_plot(explainer1.expected_value, shap_values1[0, :], x_train.iloc[0,:])

## class balance를 맞추기 위한 resampling

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
smote = SMOTE()
x_train, y_train = smote.fit_resample(x_train, y_train)

## 비지도 학습

In [ ]:
ks = range(1,50)
inertias = []

for k in ks:
    model = KMeans(n_clusters=k)
    model.fit(mobile_x)
    inertias.append(model.inertia_)

In [ ]:
plt.figure(figsize=(20,8))
# Plot ks vs inertias
plt.plot(ks, inertias, '-o')
plt.xlabel('number of clusters, k')
plt.ylabel('inertia')
plt.xticks(ks)
plt.show()

In [ ]:
# 모델링[KNN]
model = KMeans(n_clusters=5)
model.fit(mobile_x)
pred = model.predict(mobile_x)
pred = pd.DataFrame(pred, columns = ['predict'])
# 결과 보기
mobile_y.reset_index(inplace=True, drop=True)
result = pd.concat([mobile_x, mobile_y, pred], axis =1)
result.CHURN = result.CHURN.astype('int')
# 클러스터 별 고객이탈율
result.groupby('predict')['CHURN'].mean()
# 전체 평균
result['CHURN'].value_counts() / result.shape[0]

In [ ]:
# DBSCAN 모델을 만들어 봅시다.
model = DBSCAN(eps=0.1, min_samples=3)
model.fit(x)
# fitting한 후에 모델의 labels_ 값이 찾아낸 군집 종류입니다.
clusters = model.labels_
# 군집 번호 중 -1은 이상치를 의미합니다.(어느 군집에도 포함 안되는 값들!)
clusters

## 시계열 데이터

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore')
warnings.simplefilter('ignore', ConvergenceWarning)

In [ ]:
# target 시계열로 확인 
plt.figure(figsize = (20,8))
plt.plot(data['sales'])
plt.grid()
plt.show()

temp = data[-100:]
plt.figure(figsize = (20,8))
plt.plot(temp['sales'], marker ='o')
plt.grid()
plt.show()

In [ ]:
# 잔차 분석
def residual_diag(residuals, lags = 20) :

    print('* 정규성 검정(> 0.05) : ', round(spst.shapiro(residuals)[1],5))
    print('* 정상성 검정(< 0.05) : ', round(sm.tsa.stattools.adfuller(residuals)[1],5))
    print('* 자기상관성 확인(ACF, PACF)')
    fig,ax = plt.subplots(1,2, figsize = (15,5))
    plot_acf(residuals, lags = lags, ax = ax[0])
    plot_pacf(residuals, lags = lags, ax = ax[1])
    plt.show()

In [ ]:
# 날짜 타입으로 변경
data['date'] = pd.to_datetime(data['date'])
# 날짜를 인덱스로 변환
data['DT'] = data['date']
data.set_index('DT', inplace=True)
data.head()
## 날짜단위 지정하기 : freq / 인덱스 조회시, 마지막에 있는 freq 옵션
# 일단위
data.asfreq('D').head()
# 월(말)단위
data.asfreq('M').head()
# 월초 단위
data.asfreq('MS').head()
#(추가) 빠진값 찾기
temp = data.asfreq('D')
temp.isna().sum()
# 채우기
data.asfreq('D', method = 'ffill')
df = data.asfreq('D') # 일단위 데이터이므로 이걸로 지정

In [ ]:
# y 만들기
df['y'] = df['sales'].shift(-1)
display(df.head())
display(df.tail())
# 제일 마지막 행은 삭제
df.dropna(axis = 0, inplace = True)
df.tail()

In [ ]:
# 데이터 분할
# 1) x, y 나누기
# .values(넘파이 어레이)로 변환해서 저장하는 이유 ➡ 데이터 스플릿 index를 적용해서 데이터를 가져오기 위해서
target = 'y'

x = df.drop([target, 'date'], axis = 1)
y = df.loc[:, target]

# 시계열 데이터 분할
from sklearn.model_selection import TimeSeriesSplit
x.shape
# validation set size
val_size = 30
nfold = 3

tscv = TimeSeriesSplit(n_splits = nfold, test_size = val_size)
tscv

#참조
# .split을 이용하여 fold 하나씩 인덱스들을 뽑아 낼 수 있음.
for train_index, val_index in tscv.split(x):
    print("Train:", train_index, "Val:", val_index)

In [ ]:
# 모델링
# loop 돌며 모델링(cross-validation) 수행
rmse, mae, mape = [],[],[]
residuals = []
pred = []
model = LinearRegression()

for train_index, val_index in tscv.split(x):

    # 인덱스로 데이터 분할
    x_train, y_train = x.iloc[train_index], y.iloc[train_index]
    x_val, y_val = x.iloc[val_index], y.iloc[val_index]

    # 학습
    model.fit(x_train, y_train)

    # 예측
    pr = model.predict(x_val)
    pred += list(pr)

    # 평가
    rmse.append(mean_squared_error(y_val, pr, squared = False))
    mae.append(mean_absolute_error(y_val, pr))
    mape.append(mean_absolute_percentage_error(y_val, pr))

    # 잔차 : 각 fold의 결과를 리스트로 변환하여 추가
    residuals += list(y_val - pr)

In [ ]:
# 예측 결과 평가
print('RMSE : ',round(np.mean(rmse),4))
print('MAE  : ',round(np.mean(mae),4))
print('MAPE : ',round(np.mean(mape),4))

In [ ]:
# 그래프 비교
n = val_size * nfold
pred = pd.Series(pred, index = y[-n:].index)
plt.figure(figsize = (20,8))
plt.plot(y[:-n], label = 'train')
plt.plot(y[-n:], label = 'val')
plt.plot(pred, label = 'predicted')

plt.legend()
plt.grid()
plt.show()

plt.figure(figsize = (20,8))
plt.plot(y[-n:], label = 'val')
plt.plot(pred, label = 'predicted')

plt.legend()
plt.grid()
plt.show()

In [ ]:
# 평가: 잔차분석
## 시각화
plt.figure(figsize = (12,8))
plt.plot(residuals)
plt.axhline(0, color = 'r', ls = '--')
plt.axhline(np.mean(residuals), color = 'g', ls = '--')
plt.show()

## ACF, PACF(자기상관성 여부 확인)
lags = 20

fig,ax = plt.subplots(1,2, figsize = (15,5))
plot_acf(residuals, lags = lags, ax = ax[0])
plot_pacf(residuals, lags = lags, ax = ax[1])
plt.show()

## 검정
from scipy import stats
import statsmodels.api as sm
stats.shapiro(residuals)[1]  # 정규성 검정 : Shapiro-Wilk 검정
sm.tsa.stattools.adfuller(residuals)[1]  # 정상성 검정 : ADF 검정

## 시계열 모델링

In [ ]:
# 결과 시각화
def plot_model_result(y_train, y_val, pred) :
    pred = pd.Series(pred, index = y_val.index)

    # 전체 시각화
    plt.figure(figsize = (20,12))
    plt.subplot(2,1,1)
    plt.plot(y_train, label = 'train')
    plt.plot(y_val, label = 'val')
    plt.plot(pred, label = 'pred')
    plt.legend()
    plt.grid()

    plt.subplot(2,1,2)
    plt.plot(y_val, label = 'val')
    plt.plot(pred, label = 'pred')
    plt.legend()
    plt.grid()

    plt.show()

In [ ]:
# y값 살펴보기
residual_diag(y_train, lags = 30)

# ARIMA 모델링
m1_1 = sm.tsa.SARIMAX(y_train, order=(1,0,1)).fit() # ARMA
m1_2 = sm.tsa.SARIMAX(y_train, order=(1,1,1)).fit() # ARIMA

# SARIMA 모델링 : P, D, Q, m = 1,1,1,7 로 모델을 생성합시다.
m2_1 = sm.tsa.SARIMAX(y_train, order=(5,1,4), seasonal_order=(1,1,1,7)).fit()

m3_1 = sm.tsa.SARIMAX(y_train, order=(5,1,4), seasonal_order=(1,1,1,7), exog=x_train).fit()

# 평가[잔차 진단]
residuals = m1_1.resid  # y_train과 예측값 차이
residual_diag(residuals)
# 평가[AIC] ->선형 모델에서의 적합도와, feature가 과도하게 늘어나는 것을 방지하도록 설계된 통계량이 AIC 입니다.
# 값이 작을 수록 좋은 모델
# 공식 : 𝐴𝐼𝐶=−2 ln⁡(𝐿)+2𝑘 ➡ - 모델의 적합도 + 변수의 갯수
print('model1 AIC :', m1_1.aic)
# 평가[Validation]
pred = m1_1.forecast(30)   # SARIMAX 모델을 생성하고, 예측할 때는 exog=x_val 옵션이 들어가야 함. 
print('MAE :', mean_absolute_error(y_val, pred))
print('MAPE:', mean_absolute_percentage_error(y_val, pred))
# 평가[결과 시각화]
plot_model_result(y_train, y_val, pred)

In [ ]:
# 하이퍼파라미터 튜닝

# 학습
from itertools import product
# product 함수를 이용하여 값의 조합을 구성
p = [1,2,3,4,5]
q = [1,2,3,4,5]
d = [1]
iter = list(product(p,d,q))
iter

# 튜닝 
mae, aic = [],[]
for i in iter :
    model_fit = sm.tsa.SARIMAX(y_train, order=(i[0],i[1],i[2])).fit()
    pred = model_fit.forecast(30)
    mae.append( mean_absolute_error(y_val, pred))
    aic.append(model_fit.aic)
    print(i)
    
result = pd.DataFrame({'params(p,d,q)' : iter, 'mae' : mae, 'aic':aic})

display(result.loc[result['mae'] == result.mae.min()])
display(result.loc[result['aic'] == result.aic.min()])

# 가장 성능이 좋은 p, d, q 값으로 모델을 생성합니다.
m1_3 = sm.tsa.SARIMAX(y_train, order=(5,1,4)).fit()

# 평가[잔차진단]
residuals = m1_3.resid
residual_diag(residuals) # seasonallity 
# 평가[AIC]
print('model2 AIC :', m1_3.aic)
# 평가[validation]
p1 = m1_3.forecast(30)
print('MAE :', mean_absolute_error(y_val, p1))
print('MAPE:', mean_absolute_percentage_error(y_val, p1))
# 결과 시각화
plot_model_result(y_train, y_val, p1)

## 딥러닝[keras]

In [ ]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

In [ ]:
x.shape , y.shape  # 확인해볼것

* Functional

In [ ]:
@ [Linear Regression]
# 혹시 이미 그려둔 그래프가 있다면 날려줘!
keras.backend.clear_session()

# 레이어들을 사슬로 연결하 듯이 연결!
input_layer = keras.layers.Input(shape=(1,))
output_layer = keras.layers.Dense(1)(input_layer)

# 모델의 시작과 끝을 지정
model = keras.models.Model(inputs=input_layer, outputs=output_layer)

model.compile(loss = 'mse', optimizer = 'adam')

model.fit(x, y, epochs=10, verbose=1)
# 결과 출력
print(model.predict(x).reshape(-1,) )

@ [Logistic Regression]
keras.backend.clear_session()
input_layer = keras.layers.Input(shape=(1,))
output_layer = keras.layers.Dense(1, activation='sigmoid')(input_layer)
model = keras.models.Model(inputs=input_layer, outputs=output_layer)
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])
model.fit(x, y, epochs=10, verbose=1)
print(y)
print(model.predict(x).reshape(-1,) )

* Sequential

In [ ]:
@ [Linear Regression]
# 1번 청소 : 이미 만들어진 모델이 있다면 그 모델을 없애줘
keras.backend.clear_session()

# 2번 모델 선언
model = keras.models.Sequential()

# 3번 모델 블록 조립
model.add( keras.layers.Input(shape=(1,)) )
model.add( keras.layers.Dense(1) )

## 오리지널 Sequential API
# model.add( keras.layers.Dense(1, input_shape=(1,)) )

# 4번 컴파일 
model.compile(loss='mse',optimizer='adam')
model.fit(x[:15], y[:15], epochs=10, verbose=1)
print(y[15:])
print(model.predict(x[15:]))

@ [Logistic Regression]
# 혹시 이미 그려둔 그래프가 있다면 날려줘!
keras.backend.clear_session()

# model에 순차적으로 레이어를 쌓아가겠다는 의도!
model = keras.models.Sequential()

# model에 인풋 값을 받는 레이어를 넣음
model.add( keras.layers.Input(shape=(1,)) )
# model에 Dense 레이어를 넣을거야 (최초의 레이어) : weight를 곱하고, bias를 더해주는 과정
model.add( keras.layers.Dense(1, activation='sigmoid') )

# 오리지널 Sequential API
# model.add( keras.layers.Dense(1, input_shape=(1,), activation='sigmoid') )

model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics=['accuracy'])
# keras.losses.binary_crossentropy 이걸로도 가능

model.fit(x[:15], y[:15], epochs=10, verbose=1)
print(y[15:])
print(model.predict(x[15:]))

* 멀티클래스

In [ ]:
# 미래 데이터를 먼제 떼어내야하기 때문에 먼저 분리해준다.(미래데이터는 건드리지 않는다)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=.2, random_state=2022)
x_train.shape, y_train.shape
# One-Hot Encoding  (get_dummies= x에만 적용)
class_n = len(np.unique(y_train))
y_train = to_categorical(y_train, class_n)
y_test = to_categorical(y_test, class_n)

In [ ]:
@ y 확인하기
iris.target_names

# One-Hot Encoding  (get_dummies= x에만 적용)
from tensorflow.keras.utils import to_categorical 

y = to_categorical(y, 3) # 반복 실행 주의!!(계속 생성됨)
x.shape, y.shape

@ [Sequential]
keras.backend.clear_session()
model = keras.models.Sequential()
model.add(keras.layers.Input(shape=(4,)))
model.add(keras.layers.Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train,y_train,epochs=10,verbose=1)
model.predict(x_test).reshape(-1)
y.argmax(axis=1)

@ [Functional]
keras.backend.clear_session()
il = keras.layers.Input(shape=(4,))
ol = keras.layers.Dense(3,activation='softmax')(il)
model = keras.models.Model(il,ol)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')
model.fit(x_train,y_train,epochs=10, verbose=1)
pred = model.predict(x_test)
pred[:5] # 합치면 확률값 =1

* Hidden layer 추가

In [ ]:
@ [Sequential API]

keras.backend.clear_session()
model = keras.models.Sequential()
model.add(keras.layers.Input(shape=(13,)))
model.add(keras.layers.Dense(32,activation='relu'))
model.add(keras.layers.Dense(32,activation='relu'))
model.add(keras.layers.Dense(1))
model.compile(loss='mse',optimizer='adam')
model.summary()

@ [Functional]
keras.backend.clear_session()
il = keras.layers.Input(shape=(30,))
hl = keras.layers.Dense(32, activation='relu', name='hidden1')(il)
hl = keras.layers.Dense(32, activation='relu', name='hidden2')(hl)
ol = keras.layers.Dense(1,activation='sigmoid')(hl)
model = keras.models.Model(il,ol)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

@ [multi-Functional]
keras.backend.clear_session()
il = keras.layers.Input(shape=(4,))
hl = keras.layers.Dense(32, activation='relu', name='hidden1')(il)
hl = keras.layers.Dense(32, activation='relu', name='hidden2')(hl)  # 변수명 같아도 상관없음
ol = keras.layers.Dense(3,activation='softmax')(hl)
model = keras.models.Model(il,ol)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 # optimizer=keras.optimizers.Adam(0.01)-> eta
model.summary()

## 딥러닝[ANN]

In [ ]:
# 참조코드
'''
matplolib inline 명령어를 통해서
matplot으로 그리는 플롯들을 주피터 노트북 내에서 볼 수 있게 해준다.
포맷을 retina로 바꾸면 그래프의 화질이 훨씬 좋아진다.
'''
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from tensorflow.keras.backend import clear_session
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.activations import relu, softmax

from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

In [ ]:
1) 전처리
x = wine.data
y = wine.target
x.shape, y.shape
data.target_names

## reshape -> flatten하면 필요없음
train_x.shape
train_x = train_x.reshape([train_x.shape[0],-1])
test_x = test_x.reshape([test_x.shape[0],-1])
train_x.shape # 28*28
## min-max scaling
max_n, min_n = train_x.max(), train_x.min()
max_n, min_n
train_x = (train_x - min_n) / (max_n - min_n)
test_x = (test_x - min_n) / (max_n - min_n)
print(f'max : {train_x.max()} / min : {train_x.min()}')
## target feature : One-hot Encoding
from tensorflow.keras.utils import to_categorical
len_y = len(set(train_y))
train_y = to_categorical(train_y, len_y)
test_y = to_categorical(test_y, len_y)

2) 모델링
train_x.shape, train_y.shape
keras.backend.clear_session()
model = keras.models.Sequential()
model.add(keras.layers.Input(train_x.shape[1]))
model.add(keras.layers.Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(0.01),metrics='accuracy')
model.summary()

from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss',  # 무얼 관찰할지(관측대상)
                   min_delta=0,         # 최소한 나빠지지 않으면 괜찮아
                   patience=5,          # 중요! 몇번이나 참을지(거기까지 개선 안되면 멈출거야)
                   verbose=1,            
                   restore_best_weights=True)  # (반드시 사용)학습이 멈췄을 때, 최적의 가중치로 전환해줌
model.fit(train_x, train_y, 
          validation_split=0.2,  # Train data의 20%를 Validation data로!
          callbacks=[es],        # Early Stopping 적용
          verbose=1, epochs=50)

3) 예측
pred_train = model.predict(train_x)
pred_test = model.predict(test_x)
single_pred_train = pred_train.argmax(axis=1)
single_pred_test = pred_test.argmax(axis=1)
logi_train_accuracy = accuracy_score(train_y.argmax(axis=1), single_pred_train)
logi_test_accuracy = accuracy_score(test_y.argmax(axis=1), single_pred_test)
print('트레이닝 정확도 : {:.2f}%'.format(logi_train_accuracy*100))
print('테스트 정확도 : {:.2f}%'.format(logi_test_accuracy*100))

4) 확인
mnist_labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
figure = plt.figure(figsize=(20, 10))
for i, index in enumerate(np.random.choice(test_x.shape[0], size=15, replace=False)):
    ax = figure.add_subplot(3, 5, i + 1, xticks=[], yticks=[])
    # Display each image
    ax.imshow(np.squeeze(test_x[index].reshape([28,-1])), cmap='gray' )
    
    predict_index = pred_test[index].argmax(axis=0)
    true_index = test_y[index].argmax(axis=0)
    # Set the title for each image
    ax.set_title(f"{mnist_labels[predict_index]} ({mnist_labels[true_index]})",
                 color=("green" if predict_index == true_index else "red"))
    
5) 오답확인
true_false = (test_y.argmax(axis=1) == single_pred_test)
f_id = np.where(true_false == False)[0]
f_n = len(f_id)
id = f_id[rd.randrange(0,f_n)]
print(f'id = {id}' )
print(f'다음 그림은 숫자 {test_y.argmax(axis=1)[id]} 입니다.')
print(f'모델의 예측 : {single_pred_test[id]}')
print(f'모델의 카테고리별 확률 : {np.floor(pred_test[id]*100)}')
if test_y.argmax(axis=1)[id] == single_pred_test[id] :
    print('===============')
    print('정답입니다')
    print('===============')
else : 
    print('===============')
    print('틀렸어요')
    print('===============')
plt.imshow(test_x[id].reshape([28,-1]), cmap='gray')
plt.show()

6) 평가
model.evaluate(test_x, test_y)

## Modeling : multi-input & Concatenate layer

In [ ]:
1) 데이터 불러오기
iris = load_iris()
x = iris.data
y = iris.target
print(data.DESCR) 
df_x = pd.DataFrame(x, columns=iris.feature_names)
# null값 확인 필요
iris.target_names

2) train set, test set 구분하기
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(df_x, y, test_size=0.1, random_state=2022)
train_x.shape, train_y.shape, test_x.shape, test_y.shape

추가) Scaling (데이터 수치->너무 편차가 커서(min_max 필요))
from sklearn.preprocessing import MinMaxScaler
mm_scaler = MinMaxScaler()
train_x = mm_scaler.fit_transform(train_x)
test_x = mm_scaler.transform(test_x)
pd.DataFrame(train_x, columns=iris.feature_names).describe()
train_x = pd.DataFrame(train_x, columns=iris.feature_names)
test_x = pd.DataFrame(test_x, columns=iris.feature_names)

3) length끼리, width끼리
print(df_x.columns)
tr_x_l = train_x.loc[:, ['sepal length (cm)', 'petal length (cm)'] ]
tr_x_w = train_x.loc[:, ['sepal width (cm)', 'petal width (cm)'] ]
tr_x_l.shape, tr_x_w.shape
te_x_l = test_x.loc[:, ['sepal length (cm)', 'petal length (cm)'] ]
te_x_w = test_x.loc[:, ['sepal width (cm)', 'petal width (cm)'] ]

4) One-hot Encoding
from tensorflow.keras.utils import to_categorical
train_y = to_categorical(train_y, 3)
test_y = to_categorical(test_y, 3)
train_y.shape

5-1) Modeling : multi-input & Concatenate layer
# Functional API만 가능!!!!
# 1. 세션 클리어
clear_session()
# 2. 레이어 사슬처럼 엮기 : input 2개!
il_l = Input( shape=(2,) )
hl_l = Dense(2, activation=relu)(il_l)
il_w = Input( shape=(2,) )
hl_w = Dense(2, activation=relu)(il_w)
cl = Concatenate()([hl_l, hl_w])
ol = Dense(3, activation=softmax)(cl)
# 3. 모델 시작과 끝 지정
model = Model([il_l, il_w], ol)
# 4. 모델 컴파일
model.compile(loss=categorical_crossentropy, metrics=['accuracy'],
              optimizer=Adam())
model.summary()

5-2) Modeling : multi-input & Add layer
tr_x_p.shape, train_y.shape
keras.backend.clear_session()
il_s = keras.layers.Input(shape=(2,))
hl_s = keras.layers.Dense(6,activation='swish')(il_s)
il_p = keras.layers.Input(shape=(2,))
hl_p = keras.layers.Dense(6,activation='swish')(il_p)
add_l = keras.layers.Add()([hl_s, hl_p])
ol = keras.layers.Dense(3,activation='softmax')(add_l)
model = keras.models.Model([il_s,il_p],ol)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

6) 모델 시각화
from tensorflow.keras.utils import plot_model
plot_model(model, show_shapes=True)

7) 학습
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, restore_best_weights=True)
model.fit([tr_x_l, tr_x_w], train_y, validation_split=0.1, epochs=1000, verbose=1, callbacks=[es])
        # 모델에 붓는 순서 지켜야됨